# Proceso de Identificación y Gestión de Cartera de Afiliados - Capresoca EPS

**Normatividad:**  
- *Decreto 780 de 2016*: Es la ley macro que dice qué se debe hacer (pagar) y qué pasa si no se hace (mora y suspensión). 
- *Resolución 1702 de 2021*: Fue el primer manual de instrucciones detallado sobre cómo cobrar.
- *Resolución 2082 de 2016*: Es el manual de instrucciones vigente y mejorado sobre cómo deben las EPS cobrar la cartera hoy (2025).  

**Contexto:**  
Este notebook tiene como objetivo agilizar la identificación de afiliados en estado de Mora, Aviso o Sin Pagos, para que el área de Aseguramiento de Capresoca EPS realice la gestión de cobro y notificación a afiliados y empresas, conforme a la normatividad vigente.

**Fuentes de datos principales:**  
- PILA entregada por el operador a la EPS "Pila I y Pila IP" 
- PILA conciliada por ADRES  "Pila 3047"
- Maestro contributivo de afiliados de la EPS  

**Fuentes de datos secundarias:**  
- Información interna de la EPS ;
    * relaciones laborales del sistema interno.
    * Maestro del sitema interno de la EPS.
    

El proceso permite consolidar y analizar la información para facilitar la gestión y el recaudo de cartera.

# 1. Carga de librerías y configuración inicial

In [ ]:
import pandas as pd
import os
import re
from datetime import datetime
import openpyxl  # Motor recomendado para escribir archivos Excel (.xlsx)

In [ ]:
V_Periodo_Actual = "2025-06-01"
Dia = "2025-06-01"
Mora = "2025-05-01"

# 2. Fuentes de datos y rutas

In [ ]:
#rutas Capresoca 
R_MaestroAdres = r"\\Servernas\AYC2\ASEGURAMIENTO\eps_data_management\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0014072025.TXT" # Cambiar nombre
R_Relaciones_Laborales_SIE = r"\\Servernas\AYC2\ASEGURAMIENTO\eps_data_management\SIE\Aseguramiento\relaciones laborales\Reporte_Afiliados Contributivo Relaciones Laborales_2025_07_16.csv" # Cambiar nombre
R_Ms_SIE =r"\\Servernas\AYC2\ASEGURAMIENTO\eps_data_management\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2025_07_15.csv" # Cambiar nombre

R_Pila3047 = r"\\Servernas\AYC2\ASEGURAMIENTO\eps_data_management\Procesos BDUA\Contributivo\Compensación\Pila consiliada ADRES\Pila_Unificado_Con_Aportante_2018_2025.TXT"
R_Pila_I_SIE = r"\\Servernas\AYC2\ASEGURAMIENTO\eps_data_management\SIE\Pila_SIE\Pila I"
R_Pila_IP_SIE = r"\\Servernas\AYC2\ASEGURAMIENTO\eps_data_management\SIE\Pila_SIE\Pila IP"

In [ ]:
# Rutas de salida Capresoca 
R_Salida_Pila_SIE_i = r"C:\Users\osmarrincon\Downloads\Proceso.xlsx"
#R_Salida_Relaciones_Laborales = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Compensación\_Pila_SIE\SIE\Relaciones laborales.txt"

In [ ]:
import glob

# Obtener la lista de archivos .txt en la ruta especificada
file_list = glob.glob(R_Pila_I_SIE + "/*.TXT")
file_list_IP = glob.glob(R_Pila_IP_SIE + "/*.TXT")

# Leer y concatenar todos los archivos .txt en un solo dataframe
df_pila_i_sie = pd.concat((pd.read_csv(file, sep='|', encoding='ANSI') for file in file_list), ignore_index=True)
df_pila_iP_sie = pd.concat((pd.read_csv(file, sep='|', encoding='ANSI', header=None) for file in file_list_IP), ignore_index=True)
DF_MC_Adres = pd.read_csv(R_MaestroAdres, sep=',', encoding='ansi', header=None)
df_Pila_3047 = pd.read_csv(R_Pila3047, sep=',', encoding='UTF-16')
df_Relaciones_Laborales_SIE = pd.read_csv(R_Relaciones_Laborales_SIE, sep=';', encoding='ansi')
Df_SIE = pd.read_csv(R_Ms_SIE, sep=';', dtype=str, encoding='ANSI')

## 2.1. 🧾 Registro de Logs de Entrada (Trazabilidad de Fuentes)

Para garantizar la trazabilidad, reproducibilidad y control de calidad del proceso automatizado, se implementa un mecanismo de auditoría que registra las fuentes de información que alimentan el proceso de cartera.

**Objetivo:**  
Generar estructuras de resumen (`logs_3047` y `logs_pila`) que permitan identificar con precisión qué archivos o fechas alimentaron el modelo actual. Esto facilita detectar inconsistencias, regresiones en la calidad de datos, o depurar resultados históricos.

### a) **Logs de df_Pila_3047**  
El DataFrame `df_Pila_3047` (fuente ADRES) incluye una columna `nombre_Archivo`, que contiene el nombre del archivo de origen.  
- Se extraen los nombres únicos de archivo.
- Se infiere la fecha del archivo a partir del patrón `PILA_EPSC2520241001.TXT`, reconociendo el fragmento `20241001` como la fecha `01/10/2024`.
- El resultado es un DataFrame `logs_3047` con las columnas:
  - `nombre_Archivo`
  - `Fecha` (convertida a formato datetime)

### b) **Logs de PILA Interna (`Pila_I` y `Pila_IP`)**
Los DataFrames `df_pila_i_sie` y `df_pila_iP_sie` provienen de archivos `.TXT` internos, que no incluyen el nombre del archivo como columna.  
Como aún no se han asignado nombres de columnas a estas estructuras, se toma la **columna número 18** (`índice 17`, correspondiente a la `Fecha Pago` en la estructura esperada) para inferir la última fecha de actualización de cada fuente.

- `Pila_I` → Datos principales (`df_pila_i_sie`)
- `Pila_IP` → Datos adicionales (`df_pila_iP_sie`)

El resultado es un DataFrame `logs_pila` con:
- `Origen` (Pila_I o Pila_IP)
- `Fecha Máxima` (máxima fecha de pago detectada en cada fuente)

**Resultado:**  
Ambos registros se exportan en hojas separadas dentro del archivo final (`Logs_3047`, `Logs_PILA`) para facilitar su trazabilidad, validación o comparación entre ejecuciones del modelo de cartera.


In [ ]:
# --- Logs para df_Pila_3047 ---
logs_3047 = (
    df_Pila_3047[["nombre_Archivo"]]
    .dropna()
    .drop_duplicates()
    .assign(
        Fecha=lambda df: pd.to_datetime(
            df["nombre_Archivo"].str.extract(r'EPSC25(\d{8})')[0],
            format='%Y%m%d',
            errors='coerce'
        )
    )
)


# --- Logs para fuentes internas PILA_I y PILA_IP (usando la columna 17 = Fecha Pago) ---
logs_pila = pd.concat([
    pd.DataFrame({
        "Origen": "Pila_I",
        "Fecha": pd.to_datetime(df_pila_i_sie.iloc[:, 17], errors='coerce')
    }),
    pd.DataFrame({
        "Origen": "Pila_IP",
        "Fecha": pd.to_datetime(df_pila_iP_sie.iloc[:, 17], errors='coerce')
    })
], ignore_index=True)

logs_pila = (
    logs_pila
    .groupby("Origen")["Fecha"]
    .max()
    .reset_index()
    .rename(columns={"Fecha": "Fecha Máxima"})
)

# 3. Limpieza y normalización de datos
## 3.1 Limpieza de PILA IP y PILA 3047
- Reemplazo de valores
- Definicón del origen de los datos 

In [ ]:
# Reemplazar los valores 'X' por 1 en todo el dataframe df_pila_iP_sie
df_pila_iP_sie = df_pila_iP_sie.replace('X', 1)
df_Pila_3047 = df_Pila_3047.replace('X', 1)

In [ ]:
df_pila_i_sie['origen'] = 'Pila_I'
df_pila_iP_sie['origen'] = 'Pila_IP'
df_Pila_3047['origen'] = 'Pila_3047'
print(f"Número de columnas en df_pila_i_sie: {df_pila_i_sie.shape[1]}")
print(f"Número de columnas en df_pila_iP_sie: {df_pila_iP_sie.shape[1]}")
print(f"Número de columnas en df_Pila_3047: {df_Pila_3047.shape[1]}")

## 3.2. 🔗 Unificación de Fuentes de PILA Interna

En esta celda se realiza la unificación de dos fuentes internas relacionadas con los pagos PILA (`df_pila_i_sie` y `df_pila_iP_sie`). El propósito es consolidar la información de pagos reportados desde distintas instancias internas del sistema SIE, manteniendo una única base de análisis.

**Pasos realizados:**
1. Se valida que ambos DataFrames tengan la misma cantidad de columnas.
2. Se renombra `df_pila_iP_sie` para asegurar consistencia en los nombres de columnas.
3. Se concatenan ambos DataFrames en `df_pila_i_sie`.
4. Se verifica que la unificación se haya realizado correctamente.
5. Se elimina `df_pila_iP_sie` para liberar memoria.

Este proceso asegura que toda la información de pagos desde diferentes módulos del sistema quede unificada para su posterior análisis de cartera.


In [ ]:
# Mostrar la cantidad de registros antes de la unificación
print(f"Cantidad de registros en df_pila_i_sie antes de la unificación: {len(df_pila_i_sie)}")
print(f"Cantidad de registros en df_pila_iP_sie antes de la unificación: {len(df_pila_iP_sie)}")

# Validar que ambos dataframes tengan la misma cantidad de columnas
if df_pila_iP_sie.shape[1] == df_pila_i_sie.shape[1]:
    # Asignar el mismo nombre de columnas de df_pila_i_sie a df_pila_iP_sie
    df_pila_iP_sie.columns = df_pila_i_sie.columns
    
    # Unificar ambos dataframes
    df_pila_i_sie = pd.concat([df_pila_i_sie, df_pila_iP_sie], ignore_index=True)
    
    # Validar que la unificación se haya realizado correctamente
    if len(df_pila_i_sie) > len(df_pila_i_sie) - len(df_pila_iP_sie):
        print("Unificación realizada correctamente.")
    else:
        print("Error en la unificación de los dataframes.")
    
    # Eliminar el dataframe df_pila_iP_sie
    del df_pila_iP_sie
else:
    print("Los dataframes no tienen la misma cantidad de columnas. No se puede realizar la unificación.")

# Mostrar la cantidad de registros después de la unificación
print(f"Cantidad de registros después de la unificación: {len(df_pila_i_sie)}")

## 3.3🔍 Filtrado del Último Período de Pago por Aportante

Esta celda tiene como objetivo conservar únicamente el registro más reciente de cada aportante (empresa o entidad) con base en la columna `Perido Pago`. Para ello:

1. Se convierte la columna `Perido Pago` al tipo `datetime` usando el formato `%Y-%m`.
2. Se ordenan los registros por `Razón Social Aportante`, `N° Identificación Aportante` y `Perido Pago` en orden descendente.
3. Se eliminan los duplicados por aportante, manteniendo solo el período de pago más reciente.

Este paso es clave para consolidar la información y evitar duplicidades en el análisis de cartera, permitiendo identificar la última vez que cada empresa realizó aportes en PILA.


In [ ]:
df_Datos_Complementarios = df_pila_i_sie

# Mostrar la cantidad de registros antes de filtrar
print(f"Cantidad de registros antes de filtrar: {len(df_Datos_Complementarios)}")

# Convertir la columna 'Perido Pago' a tipo datetime
df_Datos_Complementarios['Perido Pago'] = pd.to_datetime(df_Datos_Complementarios['Perido Pago'], format='%Y-%m')

# Ordenar el dataframe por las columnas especificadas y por 'Perido Pago' en orden descendente
df_pila_df_Datos_Complementariosi_sie = df_Datos_Complementarios.sort_values(by=['Razón Social Aportante', 'N° Identificación Aportante', 'Perido Pago'], ascending=[True, True, False])

# Eliminar duplicados manteniendo solo los registros con el periodo máximo
df_Datos_Complementarios = df_Datos_Complementarios.drop_duplicates(subset=['Razón Social Aportante', 'N° Identificación Aportante'], keep='first')

# Mostrar la cantidad de registros después de filtrar
print(f"Cantidad de registros después de filtrar: {len(df_Datos_Complementarios)}")

In [ ]:
# Mostrar las primeras filas del dataframe
print(f"Cantidad de registros Pila SIE: {len(df_pila_i_sie)}")

print(DF_MC_Adres.columns)
print(f"Cantidad de registros MC ADRES: {len(DF_MC_Adres)}")

## 3.4. 🧾 Depuración y Selección de Variables del Maestro Contributivo ADRES

Esta celda realiza la depuración inicial del archivo maestro contributivo proveniente de ADRES (`DF_MC_Adres`). El objetivo es conservar únicamente las columnas relevantes para la identificación del afiliado y su clasificación en el régimen contributivo.

**Pasos realizados:**
1. Se seleccionan 11 columnas clave mediante índices posicionales (`columns_to_keep`), asegurando eficiencia en el manejo de estructuras de archivos sin encabezado estandarizado.
2. Se filtran los registros excluyendo aquellos con tipo de afiliado `"AF"` (Afiliado Fallecido) o con ese campo vacío.
3. Se renombran las columnas seleccionadas para facilitar la interpretación y análisis.
4. Se imprimen las columnas resultantes y la cantidad total de registros filtrados.

Esta depuración permite trabajar con una versión optimizada del maestro contributivo, útil para cruces con PILA y detección de inconsistencias en estado y afiliación.


In [ ]:
# Seleccionar solo las columnas especificadas
columns_to_keep = [
    4, 5, 6, 7, 8, 9, 17, 23, 24, 33, 41 
]

# Filtrar el dataframe para mantener solo las columnas especificadas
DF_MC_Adres = DF_MC_Adres[columns_to_keep]
DF_MC_Adres = DF_MC_Adres[
    (DF_MC_Adres[33] != "AF") | (DF_MC_Adres[33].isna())
]

# Asignar nombres a las columnas seleccionadas
DF_MC_Adres.columns = [
    'Tp_Do', 'No_Do', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento', 'Municipio', 'Estado_ADRES', 'Sisben'
]


# Mostrar las primeras filas del dataframe resultante
print(DF_MC_Adres.columns)
print(f"Cantidad de registros: {len(DF_MC_Adres)}")

## 3.5. 🧹 Eliminación de Planillas Corregidas y Selección de Variables Relevantes – PILA Interna

Esta celda tiene como objetivo depurar los registros del DataFrame `df_pila_i_sie` eliminando las planillas que fueron corregidas por los afiliados, ya que estas pueden generar duplicidad o sesgos en el análisis de cotizaciones y mora.

**Contexto técnico:**
Cuando un afiliado realiza una corrección sobre su planilla (por ejemplo, modifica los días cotizados o retira una novedad errónea), el sistema PILA genera dos planillas: la original y la corregida. Es fundamental conservar solo la última (válida), eliminando aquellas con indicador de corrección `"A"` en la columna `Correcciones`.

**Pasos realizados:**
1. Se eliminan los registros marcados como correcciones (`Correcciones = "A"`), conservando únicamente planillas válidas o sin marca.
2. Se seleccionan las columnas clave para el análisis de aportes, afiliación y origen de los datos:
   - Datos del aportante y cotizante.
   - Período y fecha de pago.
   - Ingresos y valores cotizados.
   - Indicador de origen de la información (fuente de carga).

Este paso asegura que el análisis posterior no se vea afectado por registros corregidos que podrían duplicar o distorsionar la información real de aportes.


In [ ]:
df_pila_i_sie = df_pila_i_sie[
    (df_pila_i_sie['Correcciones'] != "A") | (df_pila_i_sie['Correcciones'].isna())
]

# Seleccionar solo las columnas especificadas
columns_to_keep = [
    'Razón Social Aportante', 'N° Identificación Aportante', 'Perido Pago', 'Fecha Pago',
    'Tipo Documento Cotizante', 'N° Identificación Cotizante', 'Tipo Cotizante', 'ING', 'RET', 'Días Cotizados', 
    'Ingreso Base Cotización', 'Cotización Obligatoria', 'Número Planilla', 'origen'
]
# Filtrar el dataframe para mantener solo las columnas especificadas
df_pila_i_sie = df_pila_i_sie[columns_to_keep]

## 3.6. 🧾 Estandarización de Columnas del Archivo PILA 3047 – Conciliada ADRES

Esta celda tiene como objetivo estandarizar la estructura del archivo `df_Pila_3047`, correspondiente a la planilla PILA conciliada por ADRES, con el fin de unificar su análisis con otras fuentes de información interna (como PILA SIE).

**Pasos realizados:**
1. Se renombran las columnas originales para adoptar la misma nomenclatura utilizada en el análisis interno de PILA (`df_pila_i_sie`), garantizando compatibilidad estructural.
2. Se reorganizan las columnas en un orden lógico que facilita su comparación, consolidación y posterior cruce de datos con otras fuentes como:
   - Maestro contributivo EPS.
   - Relación laboral SIE.
   - PILA del operador.

La estandarización de esta fuente es fundamental para evitar ambigüedades, asegurar trazabilidad en los campos clave y permitir un análisis integral de la información de aportes y cartera, conforme a los lineamientos normativos de Capresoca EPS.

In [ ]:
df_Pila_3047 = df_Pila_3047.rename(columns={
    'Num_rad_planilla': 'Número Planilla',
    'perido_pago_del_aportante': 'Perido Pago',
    'fecha_pago': 'Fecha Pago',
    'Tipo_doc_cotizante': 'Tipo Documento Cotizante',
    'doc_cotizante' : 'N° Identificación Cotizante',
    'Tipo_cotizante': 'Tipo Cotizante',
    'ingreso': 'ING',
    'retiro': 'RET',
    'dias_cotizados': 'Días Cotizados',
    'ibc': 'Ingreso Base Cotización',
    'cotizacion_obligatoria':'Cotización Obligatoria',
    'Nit': 'N° Identificación Aportante',
    'Razon_Soacial': 'Razón Social Aportante'
})
print(df_Pila_3047.columns)

In [ ]:
column_order = [
    'Razón Social Aportante', 'N° Identificación Aportante', 'Perido Pago', 'Fecha Pago',
    'Tipo Documento Cotizante', 'N° Identificación Cotizante', 'Tipo Cotizante', 'ING', 'RET', 'Días Cotizados', 
    'Ingreso Base Cotización', 'Cotización Obligatoria', 'Número Planilla', 'origen'
    ]
df_Pila_3047 = df_Pila_3047[column_order]

## 3.7. 🧪 Normalización de Tipos de Datos en PILA Interna y PILA Conciliada ADRES

Esta celda realiza la conversión y estandarización de tipos de datos numéricos clave en los DataFrames `df_pila_i_sie` (fuente interna de Capresoca EPS) y `df_Pila_3047` (planilla PILA conciliada por ADRES). Esta normalización es esencial para garantizar que operaciones como filtros, comparaciones y agregaciones puedan ejecutarse sin errores de tipo o coerción.

**Campos transformados:**

- `Tipo Cotizante`: convertido a entero (`int64`) y valores nulos reemplazados por 0.
- `Ingreso Base Cotización`: convertido a número decimal (`float64`) para permitir operaciones aritméticas.
- `N° Identificación Aportante`: asegurado como número entero, reemplazando errores o vacíos.
- Indicadores de novedad (`ING`, `RET`) y `Días Cotizados`: completados con 0 y convertidos a `int64`.

**Importancia operativa:**
Esta limpieza evita errores en procesos como:
- Uniones (`merge`) basadas en identificadores.
- Cálculo de mora y días cotizados.
- Generación de reportes consolidados por empresa o afiliado.

Al aplicar esta estandarización, se fortalece la calidad y confiabilidad del análisis de cartera de acuerdo con los lineamientos técnicos de Capresoca EPS.

Finalmente, se realiza la unión vertical (`concat`) de ambas fuentes (`df_pila_i_sie` y `df_Pila_3047`), consolidando así la base de datos completa de aportes reportados tanto por el sistema interno como por la conciliación ADRES. Esta consolidación permite un análisis integral y depurado de los registros de cotización.

In [ ]:
# Add new columns to df_sin_pagos
df_pila_i_sie['Tipo Cotizante'] = df_pila_i_sie['Tipo Cotizante'].fillna(0).astype('int64')


df_Pila_3047['Ingreso Base Cotización'] = df_Pila_3047['Ingreso Base Cotización'].astype('float64')
df_Pila_3047['N° Identificación Aportante'] = pd.to_numeric(df_Pila_3047['N° Identificación Aportante'], errors='coerce').fillna(0).astype('int64')
df_Pila_3047['Tipo Cotizante'] = df_Pila_3047['Tipo Cotizante'].astype('int64')
df_Pila_3047['ING'] = df_Pila_3047['ING'].fillna(0).astype('int64')
df_Pila_3047['RET'] = df_Pila_3047['RET'].fillna(0).astype('int64')
df_Pila_3047['Días Cotizados'] = df_Pila_3047['Días Cotizados'].fillna(0).astype('int64')
df_pila_i_sie['ING'] = df_pila_i_sie['ING'].fillna(0).astype('int64')
df_pila_i_sie['RET'] = df_pila_i_sie['RET'].fillna(0).astype('int64')


In [ ]:
# Mostrar la cantidad de registros antes de la unión
print(f"Cantidad de registros en df_Pila_3047 antes de la unión: {len(df_Pila_3047)}")
print(f"Cantidad de registros en df_pila_i_sie antes de la unión: {len(df_pila_i_sie)}")

# Unir los dataframes uno debajo del otro
df_pila_i_sie = pd.concat([df_pila_i_sie, df_Pila_3047], ignore_index=True)

# Mostrar la cantidad de registros después de la unión
print(f"Cantidad de registros en df_pila_i_sie después de la unión: {len(df_pila_i_sie)}")

## 3.8. 🔄 Cruce con Maestro Contributivo ADRES y Filtro por Estado Activo

Esta celda tiene como objetivo enriquecer y depurar la base de PILA consolidada (`df_pila_i_sie`) mediante el cruce con el Maestro Contributivo ADRES (`DF_MC_Adres`). Se busca garantizar que los registros analizados correspondan únicamente a afiliados activos en ADRES y que cuenten con información complementaria útil para la segmentación futura.

**Pasos realizados:**
1. Se estandarizan los nombres de las columnas de identificación (`Tipo Documento Cotizante` y `N° Identificación Cotizante`) para permitir la unión.
2. Se cruza la información con `DF_MC_Adres`, incorporando:
   - Apellidos y nombres.
   - Ubicación geográfica (Departamento y Municipio).
   - Estado del afiliado y puntaje Sisben.
3. Se filtran únicamente los registros cuyo estado sea `"AC"` (Activo), asegurando que el análisis posterior no incluya afiliados suspendidos, retirados o sin validación ante ADRES.

Este paso cierra el proceso de limpieza y normalización, dejando los datos listos para la identificación de afiliados en mora, aviso o sin pagos.

In [ ]:
# Cambiar el nombre de las columnas especificadas
df_pila_i_sie = df_pila_i_sie.rename(columns={
    'Tipo Documento Cotizante': 'Tp_Do', 
    'N° Identificación Cotizante': 'No_Do'
})

print(f"Cantidad de registros Pila: {len(df_pila_i_sie)}")
# Realizar la unión de los dataframes
df_pila_i_sie = df_pila_i_sie.merge(DF_MC_Adres[['Tp_Do', 'No_Do', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento', 'Municipio', 'Estado_ADRES', 'Sisben']], on=['Tp_Do', 'No_Do'], how='left')
df_pila_i_sie = df_pila_i_sie[df_pila_i_sie['Estado_ADRES'].notna() & (df_pila_i_sie['Estado_ADRES'] == "AC")]


print(f"Cantidad de registros Pila: {len(df_pila_i_sie)}")
print(f"Cantidad de registros df_pila_i_sie: {len(df_pila_i_sie)}")

## 3.9. 🧮 Normalización por Agrupación: Días Cotizados, ING y RET

Este paso aplica reglas de consolidación y estandarización sobre los registros de PILA (`df_pila_i_sie`), agrupando por empresa, afiliado y período de pago. El objetivo es asegurar la consistencia interna de los indicadores antes de aplicar filtros analíticos.

**Pasos realizados:**
1. Se convierte la columna `Días Cotizados` a tipo numérico para permitir su agregación.
2. Se agrupan los registros por:
   - N° de identificación del aportante.
   - Tipo y número de documento del afiliado.
   - Período de pago.
3. Se aplica:
   - Suma total de días cotizados dentro del grupo.
   - Máximo valor para las columnas `ING` y `RET` (si alguno en el grupo tiene novedad, se refleja en todos).

**Resultado:**
Una estructura depurada donde cada grupo representa de forma homogénea la totalidad de los días cotizados y las novedades reportadas, eliminando discrepancias dentro del mismo conjunto de identificación.


In [ ]:
# Mostrar la cantidad de registros antes de filtrar
print(f"Cantidad de registros antes de filtrar: {len(df_pila_i_sie)}")

# Convertir la columna 'Días Cotizados' a tipo numérico
df_pila_i_sie['Días Cotizados'] = pd.to_numeric(df_pila_i_sie['Días Cotizados'], errors='coerce')

# Sumar los días cotizados por cada ID1
df_pila_i_sie['Días Cotizados'] = df_pila_i_sie.groupby(
    ["N° Identificación Aportante", "Tp_Do", "No_Do", 'Perido Pago']
)['Días Cotizados'].transform('sum')

# Si en la columna ING hay un 1, todos los registros del mismo grupo quedan con 1
df_pila_i_sie['ING'] = df_pila_i_sie.groupby(
    ["N° Identificación Aportante", "Tp_Do", "No_Do", 'Perido Pago']
)['ING'].transform('max')

# Si en la columna RET hay un 1, todos los registros del mismo grupo quedan con 1
df_pila_i_sie['RET'] = df_pila_i_sie.groupby(
    ["N° Identificación Aportante", "Tp_Do", "No_Do", 'Perido Pago']
)['RET'].transform('max')

# Mostrar las primeras filas del dataframe resultante
print(f"Cantidad de registros después de la agregación: {len(df_pila_i_sie)}")

## 3.10. 📅 Filtrado del Último Período Reportado por Afiliado y Aportante

Esta celda tiene como objetivo conservar únicamente el registro correspondiente al último período de pago disponible para cada afiliado, dentro de cada empresa (aportante).

**Pasos realizados:**
1. Se convierte la columna `Perido Pago` a formato fecha (`datetime`), con formato `YYYY-MM`.
2. Se agrupa por:
   - N° de identificación del aportante.
   - Tipo y número de documento del afiliado.
3. Para cada grupo, se selecciona automáticamente el registro con la fecha máxima de período reportado.
4. El DataFrame resultante contiene un solo registro por afiliado y aportante, correspondiente al período más reciente.

**Resultado:**
Una base depurada, donde se mantiene el registro más actual de cotización por afiliado. Esto evita duplicidades y garantiza que los análisis se basen en el dato más vigente del sistema.

In [ ]:
# Convertir la columna 'Perido Pago' a datetime
df_pila_i_sie['Perido Pago'] = pd.to_datetime(df_pila_i_sie['Perido Pago'], format='%Y-%m', errors='coerce')

# Para cada grupo, obtener el índice del registro con la fecha máxima
idx_max = df_pila_i_sie.groupby(['N° Identificación Aportante', 'Tp_Do', 'No_Do'])['Perido Pago'].idxmax()

# Seleccionar únicamente esos registros
df_pila_i_sie = df_pila_i_sie.loc[idx_max].reset_index(drop=True)

# Mostrar la cantidad de registros después de filtrar
print(f"Cantidad de registros después de filtrar: {len(df_pila_i_sie)}")

## 3.11. 📬 Enriquecimiento con Información de Contacto del Aportante

Esta celda tiene como objetivo complementar el DataFrame depurado de PILA (`df_pila_i_sie`) con información de contacto proveniente de los datos internos de la EPS (`df_Datos_Complementarios`), para facilitar la posterior gestión de cobro, notificación o seguimiento por parte del área de Aseguramiento.

**Pasos realizados:**
1. Se extraen del DataFrame `df_Datos_Complementarios` las columnas:
   - `Dirección de Correspondencia`
   - `Teléfono`
   - `Correo Electrónico`
2. Se realiza un cruce (`merge`) con `df_pila_i_sie`, utilizando como clave el `N° Identificación Aportante`.

**Resultado:**
Cada registro de afiliado en PILA ahora incluye datos clave de contacto de la empresa aportante, permitiendo que los procesos de gestión de cartera puedan ejecutarse de forma más eficiente.


In [ ]:
# Seleccionar solo las columnas necesarias de df_Datos_Complementarios
df_Datos_Complementarios_subset = df_Datos_Complementarios[['N° Identificación Aportante', 'Dirección de Correspondencia', 'Teléfono', 'Correo Electrónico']]

# Realizar el merge para traer las columnas a df_pila_i_sie
df_pila_i_sie = df_pila_i_sie.merge(df_Datos_Complementarios_subset, on='N° Identificación Aportante', how='left')

print(df_pila_i_sie.columns)

## 3.12. 🧾 Extracción de Afiliados Únicos para Análisis

Esta celda extrae un subconjunto con los afiliados únicos a partir del DataFrame `df_pila_i_sie`, utilizando las columnas de identificación (`Tp_Do` y `No_Do`). Este paso permite construir una vista depurada para análisis posteriores, sin duplicidad de registros por afiliado.

**Pasos realizados:**
1. Se seleccionan únicamente las columnas `Tp_Do` y `No_Do` del DataFrame de PILA.
2. Se eliminan duplicados para obtener una lista única de afiliados presentes en el consolidado.

**Resultado:**
Un DataFrame (`df_unique_aportantes`) con un registro por afiliado, que puede usarse como base para análisis posteriores de mora, avisos o estado de pagos en ADRES y SIE.


In [ ]:
# Crear un nuevo dataframe con los valores únicos de las columnas especificadas
df_unique_aportantes = df_pila_i_sie[["Tp_Do", "No_Do"]].drop_duplicates()

# Mostrar las primeras filas del nuevo dataframe
print(f"Cantidad de registros únicos: {len(df_unique_aportantes)}")

## 3.13. 🧹 Depuración de la Base de Relaciones Laborales del SIE

Esta celda tiene como objetivo limpiar y consolidar la base de relaciones laborales internas (`df_Relaciones_Laborales_SIE`) para asegurar que cada afiliado quede representado con su vínculo laboral más reciente por empresa aportante.

**Pasos realizados:**
1. Se convierte la columna `fecha_ingreso` al tipo de dato `datetime`, con el formato estándar `%Y-%m-%d`.
2. Se agrupan los registros por documento del afiliado y del aportante (`tipo_documento`, `numero_identificacion`, etc.).
3. Para cada grupo, se conserva únicamente la fecha máxima de ingreso (`fecha_ingreso` más reciente).
4. Se eliminan los registros duplicados, conservando un único vínculo laboral vigente por afiliado y aportante.

**Resultado:**
Una base de relaciones laborales limpia y lista para ser cruzada con la PILA, que garantiza integridad temporal y unicidad de los vínculos activos para análisis posteriores.


In [ ]:
print(f"Cantidad de registros realcionaes laborales #1: {len(df_Relaciones_Laborales_SIE)}")
# Convertir la columna 'fecha_ingreso' a tipo datetime
df_Relaciones_Laborales_SIE['fecha_ingreso'] = pd.to_datetime(df_Relaciones_Laborales_SIE['fecha_ingreso'], format='%Y-%m-%d')

# Asignar la fecha máxima de 'fecha_ingreso' a todo el grupo
df_Relaciones_Laborales_SIE['fecha_ingreso'] = df_Relaciones_Laborales_SIE.groupby(
    ['tipo_documento', 'numero_identificacion', 'tipo_documento_aportante', 'numero_identificacion_aportante']
)['fecha_ingreso'].transform('max')

# Eliminar duplicados manteniendo solo un registro por grupo
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.drop_duplicates(
    subset=['tipo_documento', 'numero_identificacion', 'tipo_documento_aportante', 'numero_identificacion_aportante']
)

# Mostrar las primeras filas del dataframe resultante
print(f"Cantidad de registros realcionaes laborales #2: {len(df_Relaciones_Laborales_SIE)}")

## 3.14. 🔧 Estandarización Final y Enriquecimiento de la Base de Relaciones Laborales (SIE)

En este paso se finaliza la preparación de la base de relaciones laborales del sistema SIE (`df_Relaciones_Laborales_SIE`), alineando su estructura con el resto de fuentes y añadiendo información de contacto relevante.

**Pasos realizados:**
1. Se renombran columnas clave para homogeneizar la estructura con la base de PILA.
2. Se convierten los identificadores del aportante (`N° Identificación Aportante`) a tipo `str` para asegurar la correcta unión de DataFrames.
3. Se realiza un cruce con la PILA (`df_pila_i_sie`) para añadir la columna `Correo Electrónico` del aportante a cada relación laboral.

**Resultado:**
Una base de relaciones laborales con estructura estandarizada, identificadores consistentes y campos de contacto listos para facilitar futuras notificaciones o validaciones de vínculo laboral.


In [ ]:
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.rename(columns={
    'tipo_documento': 'Tp_Do',
    'numero_identificacion': 'No_Do',
    'tipo_documento_aportante': 'Tipo Documento Aportante',
    'numero_identificacion_aportante': 'N° Identificación Aportante',
    'razon_social': 'Razón Social Aportante'
})
print(df_Relaciones_Laborales_SIE.columns)
df_Relaciones_Laborales_SIE['N° Identificación Aportante'] = df_Relaciones_Laborales_SIE['N° Identificación Aportante'].astype(str)
df_pila_i_sie['N° Identificación Aportante'] = df_pila_i_sie['N° Identificación Aportante'].astype(str)

df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.merge(df_pila_i_sie[['N° Identificación Aportante', 'Correo Electrónico']], on=['N° Identificación Aportante'], how='left')
print(df_Relaciones_Laborales_SIE.columns)

# 4. Clasificación de Afiliados: Mora, Aviso y Sin Pagos
En esta sección se identifican y clasifican los afiliados que presentan riesgo de recaudo o interrupción de cotizaciones, según los datos cruzados entre PILA, Maestro Contributivo ADRES y el sistema interno de la EPS. Esta clasificación responde a los lineamientos del Decreto 780 de 2016 y busca facilitar la gestión oportuna de cobro por parte del área de Aseguramiento.

## 4.1. ❌ Identificación de Afiliados Sin Pagos Reportados

En este paso se identifican los afiliados activos en el Maestro Contributivo ADRES que no presentan ningún registro de cotización en la base de PILA consolidada (`df_pila_i_sie`).

**Criterios aplicados:**
1. Se cruzan las cédulas (`Tp_Do`, `No_Do`) de `DF_MC_Adres` contra los afiliados presentes en PILA (`df_unique_aportantes`).
2. Se seleccionan los registros que **no están en PILA** (`_merge = "left_only"`).
3. Se filtran solo los afiliados tipo `"C"` (Cotizante) y con estado `"AC"` (activo) en ADRES.

**Resultado:**
Una lista de afiliados activos y sin evidencia de pago, que puede ser usada para alertar, notificar o priorizar en la gestión de cartera.

In [ ]:
# Filtrar los registros en DF_MC_Adres que no están en df_unique_aportantes
df_sin_pagos = DF_MC_Adres.merge(df_unique_aportantes, on=["Tp_Do", "No_Do"], how="left", indicator=True)
df_sin_pagos = df_sin_pagos[df_sin_pagos["_merge"] == "left_only"].drop(columns=["_merge"])

# Filtrar las columnas [Tp_Afiliado= 'C' y Estado_ADRES= 'AC']
df_sin_pagos = df_sin_pagos[(df_sin_pagos['Tp_Afiliado'] == 'C') & (df_sin_pagos['Estado_ADRES'] == 'AC')]

# Mostrar la cantidad de registros filtrados
print(f"Cantidad de registros sin pagos: {len(df_sin_pagos)}")

### 4.1.1. 🔗 Enriquecimiento de Afiliados Sin Pagos con Datos del SIE

Este paso complementa la base de afiliados sin pagos (`df_sin_pagos`) con información adicional proveniente de las relaciones laborales internas (`df_Relaciones_Laborales_SIE`), con el fin de mejorar la trazabilidad del vínculo laboral y la capacidad operativa de contacto para acciones de cobranza.

**Pasos realizados:**
1. Se asegura que los documentos (`No_Do`) estén en formato `str` en ambas fuentes para permitir una unión sin errores.
2. Se cruzan los datos usando como claves `Tp_Do` y `No_Do`.
3. Se añaden los siguientes campos:
   - `Tipo Documento Aportante`
   - `N° Identificación Aportante`
   - `Razón Social Aportante`
   - `fecha_ingreso` del vínculo laboral
   - `Correo Electrónico` del aportante

**Resultado:**
Una vista de afiliados sin pagos con contexto empresarial y laboral, útil para procesos de contacto, priorización y validación de posibles omisiones de pago o mora en el sistema.

In [ ]:
# Convertir 'No_Do' en df_Relaciones_Laborales_SIE a str para que coincida con df_sin_pagos
df_Relaciones_Laborales_SIE['No_Do'] = df_Relaciones_Laborales_SIE['No_Do'].astype(str)
df_sin_pagos['No_Do'] = df_sin_pagos['No_Do'].astype(str)

# Realizar el merge para traer la columna 'fecha_ingreso' a df_sin_pagos
df_sin_pagos = df_sin_pagos.merge(df_Relaciones_Laborales_SIE[['Tp_Do', 'No_Do', 'Tipo Documento Aportante', 'N° Identificación Aportante', 'Razón Social Aportante', 'fecha_ingreso', 'Correo Electrónico']], on=['Tp_Do', 'No_Do'], how='left')
print(df_sin_pagos.columns)

### 4.1.2. 🧮 Consolidación Única de Afiliados Sin Pagos

Una vez enriquecida la base `df_sin_pagos` con información laboral y de contacto, este paso tiene como objetivo asegurar que cada afiliado aparezca una sola vez en el listado, independientemente de cuántas relaciones laborales tenga registradas.

**Pasos realizados:**
1. Se eliminan duplicados utilizando como clave de unicidad la combinación `Tp_Do` + `No_Do` (tipo y número de documento del afiliado).
2. Se conserva únicamente un registro por afiliado para evitar alertas o notificaciones duplicadas.

**Resultado:**
Una base de afiliados sin pagos depurada y consolidada, con un registro único por persona, lista para priorización operativa o generación de reportes para el área de Aseguramiento.


In [ ]:
print(f"Cantidad de registros sin pagos: {len(df_sin_pagos)}")
# Dejar valores únicos en df_sin_pagos según las columnas ['Tp_Do', 'No_Do']
df_sin_pagos = df_sin_pagos.drop_duplicates(subset=['Tp_Do', 'No_Do'])

# Mostrar las primeras filas del dataframe resultante para verificar
print(f"Cantidad de registros únicos en df_sin_pagos: {len(df_sin_pagos)}")
print(df_sin_pagos.columns)

### 4.1.3. 🔁 Revalidación Final de Afiliados Sin Pagos

Luego del proceso de enriquecimiento y consolidación de `df_sin_pagos`, se realiza una revalidación cruzando nuevamente con `df_unique_aportantes` para asegurar que los afiliados seleccionados no hayan sido incorporados por error durante etapas intermedias del análisis.

**Pasos realizados:**
1. Se aseguran los formatos de las columnas `No_Do` como `string` para evitar errores de unión.
2. Se vuelve a cruzar la base de sin pagos con los afiliados presentes en PILA consolidada (`df_unique_aportantes`) para garantizar exclusividad.
3. Se aplica nuevamente el filtro de afiliados tipo `"C"` y con estado `"AC"` en ADRES.

**Resultado:**
Una validación final que garantiza que los afiliados listados en `df_sin_pagos` no tienen registros en PILA, están activos ante ADRES, y pertenecen al régimen contributivo, con un único registro por persona.


In [ ]:
# Convert No_Do to string type in both dataframes
df_sin_pagos['No_Do'] = df_sin_pagos['No_Do'].astype(str)
df_unique_aportantes['No_Do'] = df_unique_aportantes['No_Do'].astype(str)

# Now perform the merge
df_sin_pagos = df_sin_pagos.merge(df_unique_aportantes, on=["Tp_Do", "No_Do"], how="left", indicator=True)
df_sin_pagos = df_sin_pagos[df_sin_pagos["_merge"] == "left_only"].drop(columns=["_merge"])

# Filter records with required conditions
df_sin_pagos = df_sin_pagos[(df_sin_pagos['Tp_Afiliado'] == 'C') & (df_sin_pagos['Estado_ADRES'] == 'AC')]

# Show results
print(f"Cantidad de registros sin pagos: {len(df_sin_pagos)}")
print(df_sin_pagos.columns)

### 4.1.4. 🧱 Estandarización Estructural y Reorganización de Afiliados Sin Pagos

Con el fin de permitir una futura concatenación o análisis conjunto entre `df_sin_pagos` y `df_pila_i_sie`, este paso ajusta la estructura del DataFrame `df_sin_pagos` para que coincida exactamente con el esquema de columnas, tipos y orden de la base consolidada de PILA.

**Pasos realizados:**

1. **Creación de columnas faltantes**: Se agregan campos que no existen en `df_sin_pagos`, asignando valores por defecto:
   - Numéricos: `Tipo Cotizante`, `Días Cotizados`, `Ingreso Base Cotización`, `Número Planilla`, etc.
   - Categóricos: `Dirección de Correspondencia`, `Correcciones`, `origen` (establecido como `"MC_ADRES"`).
   - Fechas: `Perido Pago` y `fecha_ingreso` se inicializan como vacías (`datetime`).

2. **Conversión de tipos de datos**: Se aseguran tipos consistentes con `df_pila_i_sie`:
   - `int64` para campos numéricos.
   - `float64` para ingresos base.
   - `str` con relleno (`zfill`) para códigos de `Departamento` (2 dígitos) y `Municipio` (3 dígitos).

3. **Reorganización de columnas**: Se reordena el DataFrame `df_sin_pagos` para que el orden de las columnas sea idéntico al de `df_pila_i_sie`, facilitando la unión vertical (`concat`) o análisis conjunto.

**Resultado:**
Una versión totalmente estructurada de los afiliados sin pagos, con la misma forma, tipos y orden que la base de PILA consolidada, lista para integración, visualización o exportación operativa.

In [ ]:
# Add new columns to df_sin_pagos
df_sin_pagos['No_Do'] = df_sin_pagos['No_Do'].astype('int64')
df_sin_pagos['N° Identificación Aportante'] = df_sin_pagos['N° Identificación Aportante'].fillna(0).astype('int64')
df_sin_pagos['Tipo Cotizante'] = 0
df_sin_pagos['Tipo Cotizante'] = df_sin_pagos['Tipo Cotizante'].astype('int64')
df_sin_pagos['Dirección de Correspondencia'] = ""
df_sin_pagos['Perido Pago'] = pd.to_datetime("")
df_sin_pagos['Fecha Pago'] = ""

df_sin_pagos['ING'] = 0
df_sin_pagos['ING'] = df_sin_pagos['ING'].astype('int64')
df_sin_pagos['RET'] = 0
df_sin_pagos['RET'] = df_sin_pagos['RET'].astype('int64')
df_sin_pagos['Días Cotizados'] = 0
df_sin_pagos['Días Cotizados'] = df_sin_pagos['Días Cotizados'].astype('int64')
df_sin_pagos['Ingreso Base Cotización'] = 0
df_sin_pagos['Ingreso Base Cotización'] = df_sin_pagos['Ingreso Base Cotización'].astype('float64')
df_sin_pagos['Correcciones'] = ""
df_sin_pagos['Número Planilla'] = 0
df_sin_pagos['Número Planilla'] = df_sin_pagos['Número Planilla'].astype('int64')
df_sin_pagos['Departamento'] = df_sin_pagos['Departamento'].astype('int64')
df_sin_pagos['Departamento'] = df_sin_pagos['Departamento'].astype(str).str.zfill(2)
df_sin_pagos['Municipio'] = df_sin_pagos['Municipio'].astype('int64')
df_sin_pagos['Municipio'] = df_sin_pagos['Municipio'].astype(str).str.zfill(3)
df_sin_pagos['origen'] = "MC_ADRES"

df_pila_i_sie['Tipo Cotizante'] = df_pila_i_sie['Tipo Cotizante'].fillna(0).astype('int64')
df_pila_i_sie['ING'] = df_pila_i_sie['ING'].fillna(0).astype('int64')
df_pila_i_sie['RET'] = df_pila_i_sie['RET'].fillna(0).astype('int64')
df_pila_i_sie['Departamento'] = df_pila_i_sie['Departamento'].astype('int64')
df_pila_i_sie['Departamento'] = df_pila_i_sie['Departamento'].astype(str).str.zfill(2)
df_pila_i_sie['Municipio'] = df_pila_i_sie['Municipio'].astype('int64')
df_pila_i_sie['Municipio'] = df_pila_i_sie['Municipio'].astype(str).str.zfill(3)
df_pila_i_sie['fecha_ingreso'] = pd.to_datetime("")

In [ ]:
# Reorganizar las columnas de df_sin_pagos en el orden especificado
column_order = ['Razón Social Aportante', 'Tipo Documento Aportante', 'N° Identificación Aportante', 'Dirección de Correspondencia', 'Correo Electrónico', 'Perido Pago', 'Fecha Pago', 'Tp_Do', 'No_Do', 'Tipo Cotizante', 'ING', 'RET', 'Días Cotizados', 'Ingreso Base Cotización', 'Correcciones', 'Número Planilla', 'origen', '1A', '2A', '1N', '2N', 'Tp_Afiliado', 'Departamento', 'Municipio', 'Estado_ADRES', 'Sisben', 'fecha_ingreso']
df_sin_pagos = df_sin_pagos[column_order]

### 4.1.5. 🧩 Integración Final: Unión de Afiliados con Pagos y Sin Pagos

Una vez estructuradas ambas fuentes (`df_pila_i_sie` y `df_sin_pagos`) con el mismo esquema de columnas, se realiza la concatenación vertical de los registros, unificando en un único DataFrame todos los afiliados del régimen contributivo que se encuentran en estado activo según ADRES.

**Pasos realizados:**

1. Se imprime la cantidad de registros en cada base antes de la unión:
   - `df_pila_i_sie`: Afiliados con pagos registrados en PILA (operador o ADRES).
   - `df_sin_pagos`: Afiliados activos en ADRES sin evidencia de pagos.

2. Se concatenan ambos DataFrames utilizando `pd.concat([...], ignore_index=True)` para generar un índice limpio.

3. Se imprime la cantidad total de registros resultantes tras la integración.

**Resultado:**
Una única base maestra (`df_pila_i_sie`) que contiene:
- Afiliados con vínculo laboral y pagos identificables.
- Afiliados sin pagos, pero con vínculo y estado activo en ADRES.

Esta integración servirá de insumo para reportes finales y estrategias de intervención por parte del área de Aseguramiento de Capresoca EPS.

In [ ]:
print(f"Cantidad de registros sin pagos: {len(df_sin_pagos)}")
print(f"Cantidad de registros PAgos SIE pagos: {len(df_pila_i_sie)}")
df_pila_i_sie = pd.concat([df_pila_i_sie, df_sin_pagos], ignore_index=True)
print(f"Cantidad de registros Total: {len(df_pila_i_sie)}")

## 4.2. 🧯 Depuración Final: Exclusión de Registros Retirados y Fuera de Periodo

Después de unificar la base de afiliados con y sin pagos, se realiza un nuevo filtrado para excluir los registros que corresponden a afiliados retirados o cuya cotización pertenece al período actual (en curso), ya que estos no requieren intervención inmediata.

**Pasos realizados:**

1. Se asegura que la columna `Perido Pago` esté en formato `datetime`.
2. Se eliminan los registros con novedad de retiro (`RET = 1`), ya que estos afiliados ya no hacen parte activa del sistema.
3. Se restringe el análisis a registros con `Perido Pago` **anterior al mes actual**, o sin información en esa columna (`NaT`), asumiendo que estas omisiones también podrían representar casos sin cotización reciente.

**Variable utilizada:**
- `V_Periodo_Actual`: variable definida previamente que representa el primer día del mes actual.

**Resultado:**
Una base más precisa para análisis operativo, centrada en afiliados activos sin retiro reciente y con cotizaciones omitidas o desactualizadas, lista para priorización o intervención por parte del área de Aseguramiento.


In [ ]:
# Ensure 'Perido Pago' is datetime
print(f"Cantidad de registros df_pila_i_sie: {len(df_pila_i_sie)}")

df_pila_i_sie['Perido Pago'] = pd.to_datetime(df_pila_i_sie['Perido Pago'], errors='coerce')

# Filter out records where RET = 1
df_pila_i_sie = df_pila_i_sie[
    (df_pila_i_sie['RET'] != 1) &
    ((df_pila_i_sie['Perido Pago'] < pd.to_datetime(V_Periodo_Actual)) | (df_pila_i_sie['Perido Pago'].isna()))
]
print(f"Cantidad de registros después de filtrar RET != 1: {len(df_pila_i_sie)}")

## 4.3. 📊 Clasificación de Afiliados según Estado de Cartera

En este paso se asigna a cada afiliado un estado de cartera (`Sin Pagos`, `Mora`, `Aviso`, `Al Día`) en función de la fecha de su último período cotizado (`Perido Pago`), comparada con los umbrales definidos por la EPS.

**Variables utilizadas:**
- `Mora`: fecha límite para considerar que un afiliado ha caído en mora.
- `Dia`: fecha a partir de la cual un afiliado puede considerarse al día.

**Clasificación aplicada:**
1. Se crea la columna `Cartera` con valor por defecto `"Sin Pagos"`, para casos sin `Perido Pago`.
2. Para afiliados con fecha válida:
   - Si `Perido Pago` < `Mora`: se clasifica como `"mora"`.
   - Si `Mora` ≤ `Perido Pago` < `Dia`: se clasifica como `"Aviso"`.
   - Si `Perido Pago` ≥ `Dia`: se clasifica como `"Al Día"`.

**Resultado:**
Cada afiliado queda clasificado según su nivel de riesgo de cartera, permitiendo priorizar acciones correctivas, preventivas o informativas desde el área de Aseguramiento.

In [ ]:
# Create new column 'Cartera' based on conditions
df_pila_i_sie['Cartera'] = 'Sin Pagos'  # Default value

# Convert Mora to datetime
mora_date = pd.to_datetime(Mora)
Dia_date = pd.to_datetime(Dia)

# Update values based on conditions for non-null Perido Pago
mask = df_pila_i_sie['Perido Pago'].notna()
df_pila_i_sie.loc[mask & (df_pila_i_sie['Perido Pago'] < mora_date), 'Cartera'] = 'mora'
df_pila_i_sie.loc[mask & (df_pila_i_sie['Perido Pago'] >= mora_date), 'Cartera'] = 'Aviso'
df_pila_i_sie.loc[mask & (df_pila_i_sie['Perido Pago'] >= Dia_date), 'Cartera'] = 'Al Dia'
print(f"Cantidad de registros df_pila_i_sie: {len(df_pila_i_sie)}")

## 4.4. 🧷 Consolidación del Vínculo Laboral Vigente por Afiliado

Como preparación para reportes o análisis posteriores, se depura la base de relaciones laborales internas (`df_Relaciones_Laborales_SIE`) para conservar únicamente un registro por afiliado, correspondiente a su vínculo laboral más reciente.

**Pasos realizados:**

1. Se ordena la base por:
   - Tipo y número de documento (`Tp_Do`, `No_Do`).
   - Fecha de ingreso (`fecha_ingreso`), en orden descendente (del más reciente al más antiguo).
2. Se eliminan duplicados, conservando solo el primer registro de cada afiliado (que será el más reciente por el orden aplicado).
3. Se reinicia el índice del DataFrame resultante.

**Resultado:**
Una base consolidada con un único vínculo laboral vigente por afiliado, útil para reportes operativos, validación de empresas activas, o análisis de posibles evasores reincidentes por empresa.


In [ ]:
# Sort by date in descending order and keep first record for each ID
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.sort_values(
    by=['Tp_Do', 'No_Do', 'fecha_ingreso'], 
    ascending=[True, True, False]
)

# Drop duplicates keeping first record (which will be the one with max date due to sort)
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.drop_duplicates(
    subset=['Tp_Do', 'No_Do'], 
    keep='first'
)

# Reset the index if needed
df_Relaciones_Laborales_SIE = df_Relaciones_Laborales_SIE.reset_index(drop=True)

print(f"Cantidad de registros después de eliminar duplicados: {len(df_Relaciones_Laborales_SIE)}")

## 4.5. 📅 Validación del Rango Temporal del Período de Pago

Para asegurar que el análisis de cartera y los reportes se concentren en datos válidos y coherentes, se realiza un filtrado final sobre la columna `Perido Pago` para excluir fechas fuera de rango o erróneas.

**Pasos realizados:**

1. Se convierte la columna `Perido Pago` al formato `datetime` con estructura `dd/mm/yyyy`.
2. Se define un rango válido entre:
   - `2024-01-01` (inicio del análisis operativo).
   - `2050-12-31` (límite preventivo ante errores de digitación).
3. Se conserva:
   - Registros cuyo `Perido Pago` esté dentro del rango definido.
   - Registros con `Perido Pago` vacío (`NaT`), válidos para clasificaciones como "Sin Pagos".

**Resultado:**
Una base libre de errores por fechas anómalas, con periodos de pago consistentes y apta para reportes operativos, priorización o generación de alertas automáticas.

In [ ]:
# Convertir la columna 'Perido Pago' al formato datetime con el formato dd/mm/yyyy
df_pila_i_sie['Perido Pago'] = pd.to_datetime(df_pila_i_sie['Perido Pago'], format='%d/%m/%Y', errors='coerce')

# Definir el rango de fechas deseado
start_date = pd.to_datetime('2024-01-01')
end_date   = pd.to_datetime('2050-12-31')

# Filtrar: conservar los registros cuyo 'Perido Pago' esté vacío o esté entre las fechas definidas
mask = df_pila_i_sie['Perido Pago'].isna() | ((df_pila_i_sie['Perido Pago'] >= start_date) & (df_pila_i_sie['Perido Pago'] <= end_date))
df_pila_i_sie = df_pila_i_sie[mask]

print(f"Cantidad de registros después del filtrado: {len(df_pila_i_sie)}")

## 4.6. 🗓️ Formateo Final de Fechas para Reportes y Exportación

Para facilitar la interpretación de los datos y cumplir con los estándares visuales de presentación de reportes, se convierten las columnas de tipo fecha al formato `DD/MM/YYYY`, tradicionalmente utilizado en documentos administrativos en Colombia.

**Pasos realizados:**
1. Se define la lista `columnas_fecha` con las columnas que contienen fechas relevantes:
   - `Perido Pago`
   - `fecha_ingreso`
2. Para cada columna en esa lista:
   - Se convierte su contenido al tipo `datetime` (con manejo de errores).
   - Se transforma al formato `día/mes/año` usando `.dt.strftime('%d/%m/%Y')`.

**Resultado:**
Un DataFrame `df_pila_i_sie` con todas las fechas visibles en formato amigable para lectura humana, listo para ser exportado a Excel o utilizado en informes del área de Aseguramiento.

In [ ]:
# Lista de columnas que contienen fechas (ajusta según tu DataFrame)
columnas_fecha = ['Perido Pago', 'fecha_ingreso']  # Reemplázalas con los nombres reales

# Convertir las columnas de fecha al formato DD/MM/YYYY
for col in columnas_fecha:
    df_pila_i_sie[col] = pd.to_datetime(df_pila_i_sie[col], errors='coerce').dt.strftime('%d/%m/%Y')

## 4.7. 🧬 Normalización del NIT del Aportante para Consolidación Empresarial

En este paso se corrigen variaciones del número de identificación del aportante (`N° Identificación Aportante`) que podrían representar a una misma empresa, pero con extensiones, errores o sufijos que afectan el análisis agrupado.

**Objetivo:**  
Consolidar planillas o relaciones laborales bajo un único identificador base por empresa, especialmente cuando existen NITs como `800123456`, `800123456-1`, `800123456001`, etc.

**Pasos realizados:**

1. Se convierten todos los NITs a tipo `str` para manipulación segura.
2. Se ordena la lista única de identificaciones por longitud creciente, asumiendo que el ID base es el más corto.
3. Se crea un diccionario `id_map` que relaciona cada ID extendido con su versión base, si comparte el prefijo.
4. Se genera una nueva columna auxiliar `Id_Normalizado` aplicando ese mapeo.
5. Se eliminan duplicados por aportante y afiliado (`Id_Normalizado`, `Tp_Do`, `No_Do`), conservando el primer registro.
6. Finalmente, se elimina la columna auxiliar para dejar limpia la estructura.

**Resultado:**
Una base consolidada por empresa aportante, libre de fragmentaciones causadas por NITs con sufijos o inconsistencias, lista para análisis agregados, generación de reportes o agrupamientos confiables.

In [ ]:
# Paso 1: Convertir todos los ID a string para evitar errores
df_pila_i_sie["N° Identificación Aportante"] = df_pila_i_sie["N° Identificación Aportante"].astype(str)

# Paso 2: Obtener lista única de identificaciones ordenadas por longitud ascendente
ids_unicos = sorted(df_pila_i_sie["N° Identificación Aportante"].unique(), key=len)

# Paso 3: Crear un diccionario para mapear los IDs con sus equivalentes base (más cortos)
id_map = {}

for i, base_id in enumerate(ids_unicos):
    for other_id in ids_unicos[i+1:]:
        if other_id.startswith(base_id):
            id_map[other_id] = base_id

# Paso 4: Aplicar el mapeo al DataFrame
def mapear_id(ident):
    return id_map.get(ident, ident)

df_pila_i_sie["Id_Normalizado"] = df_pila_i_sie["N° Identificación Aportante"].apply(mapear_id)

# Paso 5: Eliminar duplicados
df_pila_i_sie = df_pila_i_sie.drop_duplicates(subset=["Id_Normalizado", "Tp_Do", "No_Do"], keep='first')

# (Opcional) Eliminar la columna auxiliar
df_pila_i_sie = df_pila_i_sie.drop(columns=["Id_Normalizado"])

## 4.8. 🧩 Enriquecimiento de Contacto desde el Maestro SIE

Después de consolidar y normalizar los registros, se identificaron afiliados cuyos campos de contacto (`Dirección de Correspondencia`, `Teléfono`, `Correo Electrónico`) estaban vacíos. Para complementar esta información, se utilizó el maestro de datos del sistema interno SIE (`Df_SIE`), cruzado por tipo y número de documento.

**Pasos realizados:**

1. Se seleccionaron los campos relevantes del archivo maestro: `tipo_documento`, `numero_identificacion`, `direccion`, `celular`, `correo_electronico`.
2. Se renombraron para facilitar la unión con `df_pila_i_sie`.
3. Se aplicó un `merge` por `Tp_Do` y `No_Do`.
4. Se actualizaron los campos vacíos del DataFrame principal únicamente cuando los valores originales estaban en blanco (`""` o `NaN`).

**Resultado:**
Una base enriquecida con mayor cobertura de información de contacto para los afiliados, lo cual facilita la **notificación oportuna** y la **gestión proactiva de cartera** por parte del área de Aseguramiento de Capresoca EPS.

In [ ]:
# --- Paso 1: Asegurar que los IDs estén en el mismo formato ---
df_pila_i_sie["Tp_Do"] = df_pila_i_sie["Tp_Do"].astype(str)
df_pila_i_sie["No_Do"] = df_pila_i_sie["No_Do"].astype(str)
Df_SIE["tipo_documento"] = Df_SIE["tipo_documento"].astype(str)
Df_SIE["numero_identificacion"] = Df_SIE["numero_identificacion"].astype(str)

# --- Paso 2: Seleccionar columnas relevantes del maestro del SIE ---
Df_SIE_contacto = Df_SIE[["tipo_documento", "numero_identificacion", "direccion", "celular", "correo_electronico"]].copy()

# --- Paso 3: Renombrar columnas para facilitar el merge ---
Df_SIE_contacto = Df_SIE_contacto.rename(columns={
    "tipo_documento": "Tp_Do",
    "numero_identificacion": "No_Do",
    "direccion": "Dirección de Correspondencia",
    "celular": "Teléfono",
    "correo_electronico": "Correo Electrónico"
})

# --- Paso 4: Realizar el merge solo si las columnas están vacías ---
campos_contacto = ["Dirección de Correspondencia", "Teléfono", "Correo Electrónico"]

for campo in campos_contacto:
    df_pila_i_sie[campo] = df_pila_i_sie[campo].fillna("")

df_pila_i_sie = df_pila_i_sie.merge(
    Df_SIE_contacto,
    on=["Tp_Do", "No_Do"],
    how="left",
    suffixes=('', '_SIE')
)

# --- Paso 5: Completar campos faltantes con valores del SIE ---
for campo in campos_contacto:
    df_pila_i_sie[campo] = df_pila_i_sie[campo].mask(df_pila_i_sie[campo] == "", df_pila_i_sie[f"{campo}_SIE"])
    df_pila_i_sie = df_pila_i_sie.drop(columns=[f"{campo}_SIE"])

## 4.9. ✉️ Validación de Correos Electrónicos

Con el fin de garantizar que las estrategias de notificación y comunicación con los afiliados sean efectivas, se implementa una función de validación que detecta correos electrónicos **inválidos operativamente**, aunque sean técnicamente válidos en cuanto a su formato.

**Motivación:**
Muchos registros contienen correos genéricos, falsos o placeholders, como:
- `actualizar@actualizar.com`
- `notiene@gmail.com`
- `a@a.com`
- `correo@correo.com`
- Correos de prueba (`ejemplo@...`, `test@...`, etc.)

Este tipo de valores impide contactar al afiliado y generan falsos positivos en indicadores de cobertura de contacto.

In [ ]:
import pandas as pd
import re

# Lista exacta de correos inválidos comunes
CORREOS_INVALIDOS_EXACTOS = {
    "a@a.com", "email@email.com", "correo@correo.com", "correo@noexiste.com", "notiene@gmail.com"
}

# Palabras clave que invalidan si aparecen solas o son parte de un patrón corto
CORREOS_PALABRAS_INVALIDAS = [
    "sincorreo", "sin_correo", "NOTINE", "no_tiene", "notiene", "actualizar", "ejemplo", "test", "prueba", "aaa", "xxxx", " "
]

# Expresión regular para validar la estructura del correo
regex_valido = re.compile(r"^[\w\.-]+@[\w\.-]+\.\w+$")

def validar_correo_operativo(correo):
    if pd.isna(correo):
        return False

    correo = str(correo).strip().lower()

    # Validación 1: estructura básica válida
    if not regex_valido.match(correo):
        return False

    # Validación 2: coincidencia exacta con correos inválidos
    if correo in CORREOS_INVALIDOS_EXACTOS:
        return False

    # Validación 3: patrones sospechosos en la parte local del correo
    local_part = correo.split('@')[0]
    for palabra in CORREOS_PALABRAS_INVALIDAS:
        if palabra in local_part:
            return False

    # Validación 4: longitud mínima total
    if len(correo) < 10:
        return False

    return True

# Aplicar validación al dataframe
df_pila_i_sie["Correo_Válido"] = df_pila_i_sie["Correo Electrónico"].apply(validar_correo_operativo)

# (Opcional) Crear subconjunto con solo los válidos
df_correos_validos = df_pila_i_sie[df_pila_i_sie["Correo_Válido"] == True]


## 4.10. 📱 Validación y Normalización de Números Telefónicos según Normativa Colombiana

Con el objetivo de asegurar la calidad y operatividad de los números telefónicos registrados en la base `df_pila_i_sie`, se implementa una función que **limpia, valida y clasifica** los datos de contacto con base en la normativa actual de marcación telefónica en Colombia.

---

### 🎯 Motivación

Durante el proceso de análisis se identificaron múltiples problemas como:

- Formatos inconsistentes: `310-2267612` → `3102267612`
- Entradas genéricas o inválidas: `"actualizar"`, `"ninguno"`, `"no tiene"`, `"0000000000"`, `"99999999"`
- Números con longitud incorrecta (menos de 10 dígitos o no normativos)

Estos errores dificultan las estrategias de **contacto, cobranza y seguimiento de cartera**.

---

### 🛠️ Reglas de Validación Aplicadas

| Tipo de Línea              | Longitud esperada | Prefijo requerido          | Ejemplo válido       |
|---------------------------|-------------------|-----------------------------|-----------------------|
| **Teléfono móvil**        | 10 dígitos        | Inicia con `3`              | `3102267612`         |
| **Línea fija nacional**   | 10 dígitos        | Inicia con `60` + indicativo (`1`, `2`, `4`, `5`, `6`, `7`, `8`) | `6012345678` |
| **Línea gratuita (toll-free)** | 11 dígitos    | Inicia con `01800`          | `018005556789`        |
| ❌ **Líneas de emergencia** | 3 dígitos         | `123`, `112`, etc. → **no válidas** | —                   |

---

### 🧹 Proceso de Limpieza y Validación

1. Se eliminan todos los caracteres no numéricos (como `-`, espacios, paréntesis).
2. Se descartan entradas con palabras clave como `"sin"`, `"no tiene"`, `"actualizar"`, etc.
3. Se evalúa si el número cumple con las reglas de longitud y prefijo según la tabla anterior.
4. Se generan dos nuevas columnas:
   - `Teléfono_Normalizado`: número limpio, solo con dígitos.
   - `Teléfono_Válido`: indicador booleano (`True`/`False`) según las reglas anteriores.

---

### ✅ Resultado

Este control de calidad permite **focalizar las estrategias de contacto** en números verificados, operativos y útiles para procesos de notificación, cobranza o contacto institucional, mejorando la eficiencia y reduciendo reprocesos.



In [ ]:
TELEFONOS_INVALIDOS = [
    "actualizar", "sin", "ninguno", "no tiene", "desconocido", "no registra", "prueba"
]

EMERGENCY_NUMBERS = {'123', '112', '119', '132', '144', '155'}

def validar_telefono_co(tel):
    if pd.isna(tel):
        return "", False

    tel_raw = str(tel).strip().lower()
    for palabra in TELEFONOS_INVALIDOS:
        if palabra in tel_raw:
            return "", False

    tel_num = re.sub(r'\D', '', tel_raw)

    if len(tel_num) == 3 and tel_num in EMERGENCY_NUMBERS:
        return tel_num, False

    if len(tel_num) == 11 and tel_num.startswith('01800'):
        return tel_num, True

    if len(tel_num) == 10:
        if tel_num.startswith('3'):
            return tel_num, True
        if tel_num.startswith('60') and tel_num[2] in {'1','2','4','5','6','7','8'}:
            return tel_num, True
        return tel_num, False

    return tel_num, False

df_pila_i_sie[['Tel_Normalizado', 'Teléfono_Válido']] = df_pila_i_sie['Teléfono'].apply(
    lambda x: pd.Series(validar_telefono_co(x))
)

# 5. 📊 Indicadores Clave de Desempeño (KPIs) del Proceso de Cartera

Con el objetivo de medir la efectividad y cobertura del proceso de generación de cartera, se calculan KPIs estratégicos por cada ejecución del modelo.

Estos indicadores permiten realizar seguimiento a la evolución del comportamiento de pagos, la calidad de los datos de contacto y la cobertura institucional del sistema de aseguramiento.

**Métricas calculadas:**

- `Fecha Generación`: Fecha en que se ejecutó el proceso.
- `Total Registros`: Total de registros analizados.
- `Sin Pagos`: Afiliados sin ningún registro de pago.
- `En Mora`: Afiliados con último pago anterior a la fecha de corte.
- `En Aviso`: Afiliados que requieren seguimiento (pero no están en mora).
- `Empresas Aportantes`: Número de empleadores distintos en la base.
- `Afiliados Únicos`: Número de afiliados identificados por documento.
- `Correos Válidos`: Afiliados con correos operativos verificados.
- `Teléfonos Válidos`: Afiliados con teléfonos operativos verificados.

El resultado se guarda en un DataFrame (`resumen_kpis`) para su posterior exportación y análisis comparativo en el tiempo.


In [ ]:
# 5. KPIs Operativos de Cartera

from datetime import datetime

# --- Fecha de generación del reporte ---
fecha_generacion = pd.to_datetime("today").strftime('%Y-%m-%d')

# --- Total general de registros ---
total_registros = len(df_pila_i_sie)

# --- Registros por tipo de cartera ---
conteo_cartera = df_pila_i_sie["Cartera"].value_counts().to_dict()
mora = conteo_cartera.get("mora", 0)
aviso = conteo_cartera.get("Aviso", 0)
sin_pagos = conteo_cartera.get("Sin Pagos", 0)

# --- Número de empresas distintas (aportantes) ---
empresas = df_pila_i_sie["N° Identificación Aportante"].nunique()

# --- Número de afiliados únicos ---
afiliados = df_pila_i_sie["No_Do"].nunique()

# --- Correos válidos ---
correos_validos = df_pila_i_sie["Correo_Válido"].sum()

# --- Teléfonos válidos ---
telefonos_validos = df_pila_i_sie["Teléfono_Válido"].sum()

# --- Crear DataFrame resumen ---
resumen_kpis = pd.DataFrame([{
    "Fecha Generación": fecha_generacion,
    "Total Registros": total_registros,
    "Sin Pagos": sin_pagos,
    "En Mora": mora,
    "En Aviso": aviso,
    "Empresas Aportantes": empresas,
    "Afiliados Únicos": afiliados,
    "Correos Válidos": correos_validos,
    "Teléfonos Válidos": telefonos_validos
}])


# 6. Generación de reportes para aseguramiento

In [ ]:
#df_pila_i_sie.to_excel(R_Salida_Pila_SIE_i, index=False, engine='openpyxl')
#df_Relaciones_Laborales_SIE.to_csv(R_Salida_Relaciones_Laborales, sep=',', index=False, encoding='ANSI')

In [ ]:
df_pila_i_sie.columns

In [134]:
import xlsxwriter

with pd.ExcelWriter(R_Salida_Pila_SIE_i, engine='xlsxwriter') as writer:
    # Exportar las hojas normales
    df_pila_i_sie.to_excel(writer, sheet_name='Cartera Consolidada', index=False)
    logs_3047.to_excel(writer, sheet_name='Logs_3047', index=False)
    logs_pila.to_excel(writer, sheet_name='Logs_PILA', index=False)

    # Escribir KPIs
    resumen_kpis.to_excel(writer, sheet_name='Resumen_KPIs', index=False)

    # Obtener el libro y la hoja de KPIs
    workbook = writer.book
    worksheet = writer.sheets['Resumen_KPIs']

    # --- Formatos personalizados ---
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'center',
        'align': 'center',
        'border': 1,
        'bg_color': '#004C97',  # Azul institucional
        'font_color': 'white'
    })

    body_format = workbook.add_format({
        'valign': 'center',
        'align': 'center',
        'border': 1,
        'bg_color': '#E9EDF5'  # Gris muy claro
    })

    number_format = workbook.add_format({
        'num_format': '#,##0',
        'valign': 'center',
        'align': 'center',
        'border': 1,
        'bg_color': '#E9EDF5'
    })

    date_format = workbook.add_format({
        'num_format': 'dd/mm/yyyy',
        'valign': 'center',
        'align': 'center',
        'border': 1,
        'bg_color': '#E9EDF5'
    })

    # --- Aplicar formato a cada celda ---
    for col_num, value in enumerate(resumen_kpis.columns):
        # Escribir encabezado con formato
        worksheet.write(0, col_num, value, header_format)

        # Aplicar formatos por tipo de dato
        if "Fecha" in value:
            fmt = date_format
        elif resumen_kpis.dtypes[value] in ['int64', 'float64']:
            fmt = number_format
        else:
            fmt = body_format

        # Aplicar a la(s) fila(s) de datos (asumiendo una sola fila de KPIs)
        worksheet.write(1, col_num, resumen_kpis.iloc[0, col_num], fmt)

        # Autoajustar ancho de columna
        col_width = max(len(str(value)), len(str(resumen_kpis.iloc[0, col_num]))) + 2
        worksheet.set_column(col_num, col_num, col_width)